In [3]:
import requests
import json

file = open('API_KEY.txt', 'r')
API_KEY = file.read()

SPORT = 'upcoming' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports

REGIONS = 'us' # uk | us | eu | au. Multiple can be specified if comma delimited

MARKETS = 'h2h' # h2h | spreads | totals. Multiple can be specified if comma delimited

ODDS_FORMAT = 'american' # decimal | american

DATE_FORMAT = 'iso' # iso | unix

odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
)

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()
    print('Number of events:', len(odds_json))
    print(json.dumps(odds_json, indent=4))

    # Check the usage quota
    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])

Number of events: 28
[
    {
        "id": "b588812ea2021bac0381f261fe424c40",
        "sport_key": "baseball_mlb",
        "sport_title": "MLB",
        "commence_time": "2024-04-12T23:40:00Z",
        "home_team": "Chicago White Sox",
        "away_team": "Cincinnati Reds",
        "bookmakers": [
            {
                "key": "wynnbet",
                "title": "WynnBET",
                "last_update": "2024-04-13T02:22:12Z",
                "markets": [
                    {
                        "key": "h2h",
                        "last_update": "2024-04-13T02:22:12Z",
                        "outcomes": [
                            {
                                "name": "Chicago White Sox",
                                "price": 1900
                            },
                            {
                                "name": "Cincinnati Reds",
                                "price": -100000
                            }
                        ]
        

In [17]:
# Event object == individual sporting event
class Event:
    def __init__(self, data):
        self.data = data
        self.sport_key = data['sport_key']
        self.id = data['id']

    def num_of_outcomes(self):
        bookmakers = self.data['bookmakers']
        for bookmaker in bookmakers:
            odds = bookmaker['markets'][0]['outcomes']

        num_outcomes = len(odds)
        return num_outcomes

    def pull_odds(self): #
        master_list = []

        bookmakers = self.data['bookmakers']
        for bookmaker in bookmakers:
            odds = bookmaker['markets'][0]['outcomes']

            for outcome in odds:

                team_name = outcome['name']
                odds_price = outcome['price']
                bookie = bookmaker['title']

                info = {
                    "team name": team_name,
                    "odds" : odds_price,
                    "bookmaker" : bookie,
                }
                
                master_list.append(info)
        return master_list
    
    def calculate_best_odds(self, list, num_outcomes): # | list = master_list 

        outcome_1_name = None
        outcome_2_name = None
        outcome_3_name = None

        outcome_1_best_odds = float("-inf")
        outcome_2_best_odds = float("-inf")
        outcome_3_best_odds = float("-inf")

        for outcome in range(len(list)):
            
            if outcome_1_name is None or outcome_1_name == list[outcome]['team name']:
                outcome_1_name = list[outcome]['team name']
                if list[outcome]['odds'] > outcome_1_best_odds:
                    outcome_1_best_odds = list[outcome]['odds']

        return [outcome_1_name, outcome_1_best_odds] 

        # Find best odds for each outcome of a matchup and return the [Team, Odds, Bookie] for each outcome

    # Calculate arbitrage | Boolean

    # Calculate arbitrage bet return | [Multiplier]

In [16]:
for i in range(len(odds_json)):
    game = Event(odds_json[i])
    master_list = game.pull_odds()
    num_outcomes = game.num_of_outcomes()
    odds = game.calculate_best_odds(master_list, num_outcomes)

    print(master_list)
    print(num_outcomes)
    print(odds)
    print()

[{'team name': 'Chicago White Sox', 'odds': 1900, 'bookmaker': 'WynnBET'}, {'team name': 'Cincinnati Reds', 'odds': -100000, 'bookmaker': 'WynnBET'}, {'team name': 'Chicago White Sox', 'odds': 3300, 'bookmaker': 'BetMGM'}, {'team name': 'Cincinnati Reds', 'odds': -10000, 'bookmaker': 'BetMGM'}]
2

[{'team name': 'Detroit Tigers', 'odds': -100000, 'bookmaker': 'FanDuel'}, {'team name': 'Minnesota Twins', 'odds': 5000, 'bookmaker': 'FanDuel'}, {'team name': 'Detroit Tigers', 'odds': -100000, 'bookmaker': 'WynnBET'}, {'team name': 'Minnesota Twins', 'odds': 1900, 'bookmaker': 'WynnBET'}, {'team name': 'Detroit Tigers', 'odds': -10000, 'bookmaker': 'BetMGM'}, {'team name': 'Minnesota Twins', 'odds': 3300, 'bookmaker': 'BetMGM'}]
2

[{'team name': 'Carolina Hurricanes', 'odds': -2500, 'bookmaker': 'Unibet'}, {'team name': 'St Louis Blues', 'odds': 850, 'bookmaker': 'Unibet'}, {'team name': 'Carolina Hurricanes', 'odds': -2100, 'bookmaker': 'DraftKings'}, {'team name': 'St Louis Blues', 'odd